## Accesibilidad a puntos de oferta agroecologica

In [1]:
import geopandas as gpd
import urbanpy as up
from tqdm import tqdm
import pandas as pd

In [2]:
tqdm.pandas() # Activate progress bar for pandas apply

In [3]:
biopoints = gpd.read_file("outputs/data/bio_points_w_wong.geojson")

In [13]:
biopoints.shape

(64, 3)

In [4]:
lima_hexs = gpd.read_file("inputs/lima_hexs9_complete.geojson")

In [5]:
biopoints.head()

,geometry
0,POINT (-77.02298 -12.13818)
1,POINT (-76.77495 -12.09439)
2,POINT (-77.02224 -12.12708)
3,POINT (-77.02562 -12.11838)
4,POINT (-77.06884 -12.09909)


In [6]:
def accessibility_analysis(hexagons, pois):
    # Calculate the Nearest Facility for each Hexagon
    hexagons['lon'] = hexagons.geometry.centroid.x
    hexagons['lat'] = hexagons.geometry.centroid.y
    pois['lon'] = pois.geometry.x
    pois['lat'] = pois.geometry.y
    
    dists, ixs = up.utils.nn_search(
        tree_features=pois[['lat', 'lon']].values,
        query_features=hexagons[['lat', 'lon']].values,
        metric='haversine'
    )
    
    hexagons["nearest_poi_ix"] = ixs
    
    # Calculate travel times and distances
    distance_duration = hexagons.progress_apply(
        lambda row: up.routing.osrm_route(
            origin=row.geometry.centroid, 
            destination = pois.iloc[row['nearest_poi_ix']]['geometry']
        ),
        result_type='expand',
        axis=1,
    )
    
    # Add columns to dataframe
    hexagons['distance_to_nearest_poi'] =  distance_duration[0] / 1000 # meters to km
    hexagons['duration_to_nearest_poi'] = distance_duration[1] / 60 # seconds to minutes
    
    custom_bins, custom_labels = up.utils.create_duration_labels(hexagons['duration_to_nearest_poi'])
    hexagons['duration_to_nearest_poi_label'] = pd.cut(hexagons['duration_to_nearest_poi'], bins=custom_bins, labels=custom_labels)
        
    return hexagons

In [7]:
up.routing.start_osrm_server("peru", "south-america", "foot")

[
    {
        "Id": "7d521f03c6e025d74065049267533b2bdda413ad026ab5bad18e96567545b31a",
        "Created": "2020-12-21T04:41:23.291870208Z",
        "Path": "osrm-routed",
        "Args": [
            "--algorithm",
            "mld",
            "/data/peru-latest.osm.pbf"
        ],
        "State": {
            "Status": "exited",
            "Running": false,
            "Paused": false,
            "Restarting": false,
            "OOMKilled": false,
            "Dead": false,
            "Pid": 0,
            "ExitCode": 0,
            "Error": "",
            "StartedAt": "2022-11-15T01:11:55.141755211Z",
            "FinishedAt": "2022-11-15T01:18:08.531800707Z"
        },
        "Image": "sha256:1cf8a5ae83ec71c8d5b0a69149b42846625eb064b36427ae6207db538506537b",
        "ResolvConfPath": "/var/lib/docker/containers/7d521f03c6e025d74065049267533b2bdda413ad026ab5bad18e96567545b31a/resolv.conf",
        "HostnamePath": "/var/lib/docker/containers/7d521f03c6e025d74065049267533

In [9]:
hexs_city_pop_access = accessibility_analysis(hexagons=lima_hexs, pois=biopoints)

/tmp/ipykernel_28033/1140997260.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hexagons['lon'] = hexagons.geometry.centroid.x
/tmp/ipykernel_28033/1140997260.py:4: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  hexagons['lat'] = hexagons.geometry.centroid.y
100%|██████████| 22991/22991 [01:51<00:00, 206.16it/s]


In [10]:
up.routing.stop_osrm_server("peru", "south-america", "foot")

UID                 PID                 PPID                C                   STIME               TTY                 TIME                CMD
root                28181               28159               22                  01:21               pts/0               00:00:49            osrm-routed --algorithm mld /data/peru-latest.osm.pbf
osrm_routing_server_south-america_peru_foot
Server was stoped succesfully


In [11]:
hexs_city_pop_access["duration_to_nearest_poi_label"] = hexs_city_pop_access["duration_to_nearest_poi_label"].astype("str")

In [12]:
hexs_city_pop_access.to_file("outputs/lima_hexs9_distance_to_bio.geojson")